# sentence encoder


Based on [https://towardsdatascience.com/text-classification-in-spark-nlp-with-bert-and-universal-sentence-encoders-e644d618ca32](https://towardsdatascience.com/text-classification-in-spark-nlp-with-bert-and-universal-sentence-encoders-e644d618ca32)




In [1]:
import com.johnsnowlabs.nlp.SparkNLP
import com.johnsnowlabs.nlp.annotator._
import com.johnsnowlabs.nlp.base._
import com.johnsnowlabs.ml.tensorflow.TensorflowBert
import org.apache.spark.ml.Pipeline
import org.apache.spark.sql.types._
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions.{udf,to_timestamp}
import org.apache.spark.ml.feature.QuantileDiscretizer

val dataDir = "../../data/recsys2020"

In [2]:
var df = spark.read.parquet(dataDir + "/training1m.parquet")
df

[text_tokens: array<string>, hashtags: array<string> ... 22 more fields]

In [9]:
// Convert targets
val udf_has_engagement = udf[Boolean, IntegerType](_ != null)
val target_df = df.withColumn("has_retweet", udf_has_engagement('retweet_timestamp))
target_df.head()

[WrappedArray([CLS], RT, @, F, ##bro, ##glia, :, Después, de, analiza, ##r, ex, ##haus, ##tiva, ##mente, el, mercado, con, ##clu, ##yo, que, los, biz, ##co, ##chi, ##tos, ale, ##m, de, que, ##so, son, lo, mejor, q, nos, dio, entre, ríos, h, [UNK], [SEP]),null,FB6304C97F6CC05AF0ADBBCD328CE572,null,null,null,Retweet,06D61DCBBE938971E1EA0C38BD9B5446,1581420802,42276CF9E61425149BD30B3312C76E20,610,215,false,1281271512,D0EA9DDFE93EDA782D94184BDA6F0A41,841,270,false,1362976270,false,null,null,null,1581428582,false]

In [7]:
// reduce workload while experimenting
df = df.limit(100)
df

[text_tokens: array<string>, hashtags: array<string> ... 22 more fields]

In [10]:
val text_df = df.withColumn("sample_text", lit("Hello World."))
text_df.head()

In [8]:
val udf_tweet = udf[String, Array[String]](_.mkString(" "))

// filtering
// .filterNot(token => List("[CLS]","[UNK]","[SEP]","UNKN").contains(token))
// folding instead of mkstring
// .foldLeft(token => )
val converted_df = df.withColumn("sentence", udf_tweet('text_tokens))
converted_df.head()

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$1: (array<string>) => string)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$24.apply(RDD.scala:858)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$anonfun$apply$24.apply(RDD.scala:858)

In [3]:
// Define stages
val use = UniversalSentenceEncoder.pretrained().setInputCols(Array("text_tokens")).setOutputCol("tweet_embeddings")
val fin = new EmbeddingsFinisher()
      .setInputCols("tweet_embeddings")
      .setOutputCols("finished_tweet_embeddings")
      .setOutputAsVector(true)
      .setCleanAnnotations(false)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
Download done! Loading the resource.


In [6]:
val sent_emb = new SentenceEmbeddings().setInputCols(Array("text_tokens"))

java.lang.IllegalArgumentException: requirement failed: setInputCols in SENTENCE_EMBEDDINGS_a0b123625945 expecting 2 columns. Provided column amount: 1. Which should be columns from the following annotators: document, word_embeddings

In [4]:
val use_clf_pipeline = new Pipeline().setStages(Array(use, fin))

In [5]:
val use_transformed = use_clf_pipeline.fit(df).transform(df)
use_transformed

java.lang.IllegalArgumentException: requirement failed: Wrong or missing inputCols annotators in UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec.

Current inputCols: text_tokens. Dataset's columns:
(column_name=text_tokens,is_nlp_annotator=false)
(column_name=hashtags,is_nlp_annotator=false)
(column_name=tweet_id,is_nlp_annotator=false)
(column_name=present_media,is_nlp_annotator=false)
(column_name=present_links,is_nlp_annotator=false)
(column_name=present_domains,is_nlp_annotator=false)
(column_name=tweet_type,is_nlp_annotator=false)
(column_name=language,is_nlp_annotator=false)
(column_name=tweet_timestamp,is_nlp_annotator=false)
(column_name=engaged_with_user_id,is_nlp_annotator=false)
(column_name=engaged_with_user_follower_count,is_nlp_annotator=false)
(column_name=engaged_with_user_following_count,is_nlp_annotator=false)
(column_name=engaged_with_user_is_verified,is_nlp_annotator=false)
(column_name=engaged_with_user_account_creation,is_nlp_annotator=false)
(column_name=engaging_user_id